In [1]:
!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch torchaudio torchvision torchtext torchdata

Found existing installation: torch 2.0.1
Uninstalling torch-2.0.1:
  Successfully uninstalled torch-2.0.1
Found existing installation: torchvision 0.15.2
Uninstalling torchvision-0.15.2:
  Successfully uninstalled torchvision-0.15.2


  Using cached torch-2.0.1-cp310-cp310-win_amd64.whl (172.3 MB)
     ---------------------------------------- 2.1/2.1 MB 2.2 MB/s eta 0:00:00
  Using cached torchvision-0.15.2-cp310-cp310-win_amd64.whl (1.2 MB)
     ---------------------------------------- 1.9/1.9 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 2.2 MB/s eta 0:00:00


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0946,  0.0431,  0.1396,  0.1420, -0.2541, -0.2606,  0.2498, -0.4687,
         -0.0752, -0.2630, -0.1513, -0.2782, -0.2184, -0.0205,  0.4997,  0.4213,
         -0.1013,  0.4841, -0.7744,  0.2528],
        [-0.2099,  0.0964, -0.1477,  0.1913,  0.0099, -0.3913,  0.0834, -0.4765,
         -0.0870, -0.0964, -0.4059, -0.3565, -0.3596,  0.4010,  0.4275,  0.2543,
          0.0936,  0.1493, -0.4564,  0.6049],
        [ 0.0083,  0.0820,  0.1992,  0.2503,  0.1290, -0.1265, -0.0133, -0.2109,
         -0.2222, -0.2463, -0.3603, -0.3863, -0.3039,  0.6192,  0.4205,  0.5489,
         -0.1169,  0.3615, -1.1810,  0.1544]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0946, 0.0431, 0.1396, 0.1420, 0.0000, 0.0000, 0.2498, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4997, 0.4213, 0.0000, 0.4841,
         0.0000, 0.2528],
        [0.0000, 0.0964, 0.0000, 0.1913, 0.0099, 0.0000, 0.0834, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.4010, 0.42

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0171, -0.0326,  0.0245,  ...,  0.0170, -0.0199, -0.0051],
        [ 0.0149,  0.0138,  0.0213,  ..., -0.0086,  0.0210, -0.0221]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0276, 0.0068], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0200, -0.0029, -0.0435,  ..., -0.0411,  0.0290, -0.0217],
        [ 0.0378,  0.0271, -0.0087,  ..., -0.0006, -0.0202,  0.0433]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si